In [1]:
import numpy as np
import os
from collections import OrderedDict
from scipy.io import wavfile

In [2]:
basedir = '/Volumes/Elements/TIDIGITS/'
mtr = 'CD1/TIDIGITS/TRAIN/MAN/'
ftr = 'CD1/TIDIGITS/TRAIN/WOMAN/'
mte = 'CD2/TIDIGITS/TEST/MAN/'
fte = 'CD2/TIDIGITS/TEST/WOMAN/'

old_separator = '/'
new_separator = '\\'

D = {'train': {'m': {}, 'f': {}}, 'test': {'m': {}, 'f': {}}}
for path, task, gender in zip([mtr, ftr, mte, fte], ['train', 'train', 'test', 'test'], ['m', 'f', 'm', 'f']):
    _list = [i for i in os.listdir(basedir + path)if len(i) == 2]
    for spk in _list:
        _wavs = [i for i in os.listdir(basedir + path + spk + '/') if "WAV" in i]
        D[task][gender][spk] = [path + spk + '/' + i for i in _wavs]

to_digit = {'O': 0,'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'Z': 0}

In [3]:
n_samples = 6000
with open('train_set.txt', 'w') as wavs:
    for i in range(n_samples):
        rnd_gnd1 = np.random.choice(['m', 'f'])
        rnd_spk1 = np.random.choice(D['train'][rnd_gnd1].keys())
        rnd_wav1 = np.random.choice(D['train'][rnd_gnd1][rnd_spk1])
        
        rnd_gnd2 = np.random.choice(['m', 'f'])
        rnd_spk2 = np.random.choice(D['train'][rnd_gnd2].keys())
        rnd_wav2 = np.random.choice(D['train'][rnd_gnd2][rnd_spk2])
        
        ann1 = "".join([str(to_digit[i]) for i in rnd_wav1.split('/')[-1][:-5]])
        ann2 = "".join([str(to_digit[i]) for i in rnd_wav2.split('/')[-1][:-5]])
        nn = rnd_spk1 + ann1 + rnd_spk2 + ann2
        wavs.write(rnd_wav1.replace(old_separator, new_separator) + ' ' + rnd_wav2.replace(old_separator, new_separator) + ' ' + ann1 + ' ' + ann2 + ' ' + nn +'\n')
        
        
    

In [4]:
n_samples = 2000
with open('test_set.txt', 'w') as wavs:
    for i in range(n_samples):
        rnd_gnd1 = np.random.choice(['m', 'f'])
        rnd_spk1 = np.random.choice(D['test'][rnd_gnd1].keys())
        rnd_wav1 = np.random.choice(D['test'][rnd_gnd1][rnd_spk1])
        
        rnd_gnd2 = np.random.choice(['m', 'f'])
        rnd_spk2 = np.random.choice(D['test'][rnd_gnd2].keys())
        rnd_wav2 = np.random.choice(D['test'][rnd_gnd2][rnd_spk2])
        
        ann1 = "".join([str(to_digit[i]) for i in rnd_wav1.split('/')[-1][:-5]])
        ann2 = "".join([str(to_digit[i]) for i in rnd_wav2.split('/')[-1][:-5]])
        
        nn = rnd_spk1 + ann1 + rnd_spk2 + ann2
        
        wavs.write(rnd_wav1.replace(old_separator, new_separator) + ' ' + rnd_wav2.replace(old_separator, new_separator) + ' ' + ann1 + ' ' + ann2 + ' ' + nn + '\n')
        